In [3]:
from faker import Faker
import mysql.connector
import random
from datetime import date, timedelta


DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': 'Sheela@1981',
    'port': 3306,
    'database': 'faker_db'
}

fake = Faker('en_US')


def create_tables():
    connection = mysql.connector.connect(**DB_CONFIG)
    cursor = connection.cursor()
    
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS Students (
            student_id INT PRIMARY KEY,
            name VARCHAR(100),
            age INT,
            gender VARCHAR(20),
            email VARCHAR(100),
            phone VARCHAR(50),
            enrollment_year INT,
            course_batch VARCHAR(50),
            city VARCHAR(100),
            graduation_year INT
        );
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS Programming (
            programming_id INT PRIMARY KEY,
            student_id INT,
            language VARCHAR(50),
            problems_solved INT,
            assessments_completed INT,
            mini_projects INT,
            certifications_earned INT,
            latest_project_score INT,
            FOREIGN KEY (student_id) REFERENCES Students(student_id)
        );
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS SoftSkills (
            soft_skill_id INT PRIMARY KEY,
            student_id INT,
            communication INT,
            teamwork INT,
            presentation INT,
            leadership INT,
            critical_thinking INT,
            interpersonal_skills INT,
            FOREIGN KEY (student_id) REFERENCES Students(student_id)
        );
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS Placements (
            placement_id INT PRIMARY KEY,
            student_id INT,
            mock_interview_score INT,
            internships_completed INT,
            placement_status VARCHAR(20),
            company_name VARCHAR(100),
            placement_package DECIMAL(10,2),
            interview_rounds_cleared INT,
            placement_date DATE,
            FOREIGN KEY (student_id) REFERENCES Students(student_id)
        );
    """)

    connection.commit()
    cursor.close()
    connection.close()


def generate_fake_students(num_students, start_student_id=1):
    students_data = []
    genders = ['Male', 'Female', 'Non-binary']
    course_batches = ['DS-Batch-A-2023', 'DA-Batch-B-2023', 'AI-Batch-C-2023',
                      'CD-Batch-A-2024', 'WD-Batch-B-2024', 'ML-Batch-C-2024']

    for i in range(num_students):
        student_id = start_student_id + i
        enrollment_year = fake.random_int(min=2020, max=2024)

        student = {
            'student_id': student_id,
            'name': fake.name(),
            'age': fake.random_int(min=21, max=35),
            'gender': random.choice(genders),
            'email': fake.unique.email(),
            'phone': fake.phone_number(),
            'enrollment_year': enrollment_year,
            'course_batch': random.choice(course_batches),
            'city': fake.city(),
            'graduation_year': enrollment_year + random.randint(1, 3)
        }
        students_data.append(student)
    return students_data

def generate_fake_programming_data(student_ids, start_programming_id=1):
    programming_data = []
    programming_languages = ['Python', 'SQL', 'Java', 'C++', 'JavaScript', 'R', 'Go']

    for i, student_id in enumerate(student_ids):
        programming = {
            'programming_id': start_programming_id + i,
            'student_id': student_id,
            'language': random.choice(programming_languages),
            'problems_solved': fake.random_int(min=20, max=300),
            'assessments_completed': fake.random_int(min=1, max=15),
            'mini_projects': fake.random_int(min=0, max=7),
            'certifications_earned': fake.random_int(min=0, max=4),
            'latest_project_score': fake.random_int(min=50, max=100)
        }
        programming_data.append(programming)
    return programming_data

def generate_fake_soft_skills_data(student_ids, start_soft_skill_id=1):
    soft_skills_data = []

    for i, student_id in enumerate(student_ids):
        soft_skill = {
            'soft_skill_id': start_soft_skill_id + i,
            'student_id': student_id,
            'communication': fake.random_int(min=60, max=100),
            'teamwork': fake.random_int(min=60, max=100),
            'presentation': fake.random_int(min=60, max=100),
            'leadership': fake.random_int(min=50, max=95),
            'critical_thinking': fake.random_int(min=65, max=100),
            'interpersonal_skills': fake.random_int(min=60, max=100)
        }
        soft_skills_data.append(soft_skill)
    return soft_skills_data

def generate_fake_placements_data(student_ids, start_placement_id=1):
    placements_data = []
    placement_statuses = ['Placed', 'Ready', 'Not Ready']

    for i, student_id in enumerate(student_ids):
        mock_interview_score = fake.random_int(min=40, max=95)
        internships_completed = fake.random_int(min=0, max=3)
        placement_status = random.choices(placement_statuses, weights=[0.6, 0.3, 0.1])[0]

        company_name = None
        placement_package = None
        interview_rounds_cleared = fake.random_int(min=0, max=2)
        placement_date = None

        if placement_status == 'Placed':
            company_name = fake.company()
            placement_package = round(random.uniform(30000, 150000), 2)
            interview_rounds_cleared = fake.random_int(min=2, max=5)
            start_date = date(date.today().year - 1, 1, 1)
            end_date = date.today()
            placement_date = fake.date_between(start_date=start_date, end_date=end_date)

        placement = {
            'placement_id': start_placement_id + i,
            'student_id': student_id,
            'mock_interview_score': mock_interview_score,
            'internships_completed': internships_completed,
            'placement_status': placement_status,
            'company_name': company_name,
            'placement_package': placement_package,
            'interview_rounds_cleared': interview_rounds_cleared,
            'placement_date': placement_date
        }
        placements_data.append(placement)
    return placements_data


def insert_data_into_db(table_name, data_list, columns):
    connection = None
    try:
        connection = mysql.connector.connect(**DB_CONFIG)
        cursor = connection.cursor()

        placeholders = ', '.join(['%s'] * len(columns))
        columns_str = ', '.join(columns)
        sql_insert_query = f"INSERT INTO {table_name} ({columns_str}) VALUES ({placeholders})"

        data_to_insert = [tuple(item[col] for col in columns) for item in data_list]
        cursor.executemany(sql_insert_query, data_to_insert)
        connection.commit()
        print(f"✅ Inserted {cursor.rowcount} records into '{table_name}'.")

    except mysql.connector.Error as err:
        print(f"Error inserting into '{table_name}': {err}")
        if connection:
            connection.rollback()
    finally:
        if connection and connection.is_connected():
            cursor.close()
            connection.close()


if __name__ == "__main__":
    create_tables()

    num_students = 250
    print(f"\n🎓 Generating {num_students} student records...")
    students_data = generate_fake_students(num_students)
    student_ids = [s['student_id'] for s in students_data]

    insert_data_into_db("Students", students_data,
                        ['student_id', 'name', 'age', 'gender', 'email', 'phone',
                         'enrollment_year', 'course_batch', 'city', 'graduation_year'])

    programming_data = generate_fake_programming_data(student_ids)
    insert_data_into_db("Programming", programming_data,
                        ['programming_id', 'student_id', 'language', 'problems_solved',
                         'assessments_completed', 'mini_projects', 'certifications_earned', 'latest_project_score'])

    soft_skills_data = generate_fake_soft_skills_data(student_ids)
    insert_data_into_db("SoftSkills", soft_skills_data,
                        ['soft_skill_id', 'student_id', 'communication', 'teamwork',
                         'presentation', 'leadership', 'critical_thinking', 'interpersonal_skills'])

    placements_data = generate_fake_placements_data(student_ids)
    insert_data_into_db("Placements", placements_data,
                        ['placement_id', 'student_id', 'mock_interview_score', 'internships_completed',
                         'placement_status', 'company_name', 'placement_package',
                         'interview_rounds_cleared', 'placement_date'])

    print("\n✅ All data generated and inserted successfully!")



🎓 Generating 250 student records...
✅ Inserted 250 records into 'Students'.
✅ Inserted 250 records into 'Programming'.
✅ Inserted 250 records into 'SoftSkills'.
✅ Inserted 250 records into 'Placements'.

✅ All data generated and inserted successfully!


In [1]:
A=10